In [1]:
import random
import sys

from pipeline import experiment_pipeline
from preprocessing import load_data
from sklearn import model_selection
random.seed(10)

arguments = sys.argv

arguments = [0, '/home/tristan/Desktop/Repos/DreamHF']
ROOT = arguments[1]

print("Loading the data...")
pheno_df_train, pheno_df_test, readcounts_df_train, readcounts_df_test = load_data(
    ROOT)


/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: RuntimeWarning: sklearn.tree._criterion.Criterion size changed, may indicate binary incompatibility. Expected 328 from C header, got 528 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: sklearn.tree._splitter.Splitter size changed, may indicate binary incompatibility. Expected 1160 from C header, got 1360 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: sklearn.tree._criterion.ClassificationCriterion size changed, may indicate binary incompatibility. Expected 1168 from C header, got 1368 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: sklearn.tree._criterion.RegressionCriterion size changed, may indicate binary incompatibility. Ex

Loading the data...


In [2]:
import os
import pathlib

import numpy as np
import pandas as pd
import sklearn
from sksurv.metrics import concordance_index_censored

import wandb
from model_evaluation import evaluate_model
from preprocessing import CLINICAL_COVARIATES, Salosensaari_processing, clr_processing

sklearn.set_config(transform_output="pandas")

In [3]:
processing = 'MI_clr'
clinical_covariates=CLINICAL_COVARIATES
n_taxa = 50

if processing == 'Salosensaari':
    X_train, X_test, y_train, y_test, test_sample_ids = Salosensaari_processing(
        pheno_df_train, pheno_df_test, readcounts_df_train, readcounts_df_test, clinical_covariates
    )
elif processing == 'MI_clr':
    ## Feature selection
    X_train, X_test, y_train, y_test, test_sample_ids = clr_processing(
        pheno_df_train, pheno_df_test, readcounts_df_train, readcounts_df_test, clinical_covariates,  n_taxa)


In [4]:
# importing metrics
import types

import numpy as np
from optuna import create_study
from optuna.samplers import TPESampler
from scipy.stats import randint, uniform
from sklearn import model_selection
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.utils import estimator_html_repr
from sklearn.utils.validation import check_is_fitted
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis, IPCRidge
from sksurv.metrics import concordance_index_censored
from xgbse import XGBSEStackedWeibull
from xgbse.converters import convert_y
from xgbse.metrics import concordance_index

from xgboost_wrapper import XGBSurvival


In [5]:
from survival_models import sksurv_model

In [6]:
class sksurv_gbt_optuna(sksurv_model):
    def __init__(self):
        super().__init__()

        # Optuna
        self.RS = 124  # random state
        # XGBoost
        self.EARLY_STOPPING_ROUNDS = 50
        self.MULTIVARIATE = True

        self.sampler = TPESampler(seed=self.RS, multivariate=self.MULTIVARIATE)

        self.estimator = GradientBoostingSurvivalAnalysis()

        self.pipeline = self.create_pipeline()

    def cross_validation(self, X_train, y_train, n_iter):
        self.N_TRIALS = n_iter

        study = create_study(direction="maximize", sampler=self.sampler)
        study.optimize(
            lambda trial: self.objective(
                trial,
                X_train,
                y_train,
                n_jobs=-1,
            ),
            n_trials=self.N_TRIALS,
            n_jobs=-1,
        )
        self.optimal_hp = study.best_params
        self.pipeline.set_params(**self.optimal_hp)
        self.pipeline = self.pipeline.fit(X_train, y_train)
        return self

    def objective(
        self,
        trial,
        X_train,
        y_train,
        n_jobs=-1,
    ):

        params = {
            "reduce_dim": trial.suggest_float("reduce_dim", 0.95, 1, log = False),
            "estimator__learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.4, log=False),
            "estimator__max_depth": trial.suggest_int("max_depth", 2, 6),
            "estimator__loss": "coxph",
            "estimator__n_estimators": trial.suggest_int("n_estimators", 100, 350),
            "estimator__min_samples_split":  trial.suggest_int("min_samples_split", 2, 6),
            "estimator__min_samples_leaf":  trial.suggest_int("min_samples_leaf", 1, 10),
            "estimator__subsample": trial.suggest_float("subsample", 0.4, 0.8, log=False),
            "estimator__max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 2, 30),
            "estimator__dropout_rate": trial.suggest_float("dropout_rate", 0, 1, log=False)
        }
        self.pipeline.set_params(**params)
        score = model_selection.cross_val_score(
            self.pipeline, X_train, y_train, n_jobs=-1, cv=3)
        accuracy = score.mean()
        return accuracy


In [15]:

model =sksurv_gbt_optuna()
n_iter = 10
model = model.cross_validation(X_train, y_train, n_iter)


/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-01-02 18:00:10,595] A new study created in memory with name: no-name-0a37cb0c-5341-42e5-ae5d-502fafdb60ba
/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains PCA(n_components=0.98) which is of type PCA.
  warnings.warn(message)
/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains PCA(n_components=0.95) which is of type PCA.
  warnings.warn(message)
[W 2023-01-02 18:00:10

ValueError: CategoricalDistribution does not support dynamic value space.

In [60]:
class xgb_optuna(candidate_model):
    def __init__(self):
        super().__init__()
                # Optuna
        self.RS = 124  # random state
        # XGBoost
        self.EARLY_STOPPING_ROUNDS = 50
        self.MULTIVARIATE = True

        self.sampler = TPESampler(seed=self.RS, multivariate=self.MULTIVARIATE)
        self.base_params = {
            "objective": "survival:aft",
            "eval_metric": "aft-nloglik",
            "aft_loss_distribution": "normal",
            "aft_loss_distribution_scale": 1.20,
            "tree_method": "hist",
            "booster": "dart",
            "learning_rate": 0.032833188230587194,
            "max_depth": 10,
            "subsample": 0.61828926669036,
            "alpha": 0.012673256334558281,
            "lambda": 6.468264510932119,
            "verbosity": 0,
            "n_estimators": 10000,
            "seed": self.RS,
        }

        self.estimator = XGBSurvival(self.base_params, num_boost_round=10000)
        self.pipeline = self.create_pipeline()

    def cross_validation(self, X_train, y_train, n_iter):
        self.N_TRIALS = n_iter

        study = create_study(direction="maximize", sampler=self.sampler)
        study.optimize(
            lambda trial: self.objective(
                trial,
                X_train,
                y_train,
                n_jobs=-1,
            ),
            n_trials=self.N_TRIALS,
            n_jobs=-1,
        )
        self.optimal_hp = study.best_params
        self.pipeline.set_params(**self.optimal_hp)
        self.pipeline = self.pipeline.fit(X_train, y_train)
        return self

    def objective(
        self,
        trial,
        X_train,
        y_train,
        n_jobs=-1,
    ):
        # XGBoost parameters

        xgb_params = {
            "estimator__objective": "survival:aft",
            "estimator__eval_metric": "aft-nloglik",
            "estimator__aft_loss_distribution": "normal",
            "estimator__aft_loss_distribution_scale": trial.suggest_float('aft_loss_distribution_scale', 0.1, 10.0, log=True),
            "estimator__tree_method": "hist",
            "estimator__learning_rate": trial.suggest_float("learning_rate", 1e-2, 1, log=True),
            "estimator__max_depth": trial.suggest_int("max_depth", 2, 12),
            "estimator__booster": "dart",
            "estimator__subsample": trial.suggest_float("subsample", 0.4, 0.8, log=False),
            "estimator__alpha": trial.suggest_float("alpha", 0.01, 10.0, log=True),
            "estimator__lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
            "estimator__gamma": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
        }
        
        #pipeline_params = {"reduce_dim": trial.suggest_categorical("reduce_dim", ['passthrough', PCA(0.95), PCA(0.98)])}
           
        self.estimator = XGBSurvival(xgb_params, num_boost_round=10000)
        self.pipeline = self.create_pipeline()
        #self.pipeline.set_params(**pipeline_params)
        
        score = model_selection.cross_val_score(self.pipeline, X_train, y_train, n_jobs=-1, cv=3)
        accuracy = score.mean()
        return accuracy    

    def risk_score(self, X_test):
        risk_score = xgb_risk_score(self, X_test)
        return risk_score


In [65]:
model = xgb_optuna()
model.pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

In [63]:
model = xgb_optuna()
model = model.cross_validation(X_train, y_train, 10)

/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:281: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-01-02 17:06:08,425] A new study created in memory with name: no-name-97903072-77bb-4ced-8248-b216a5f5f8ca
[W 2023-01-02 17:08:26,822] Trial 2 failed because of the following error: KeyError('handle')
Traceback (most recent call last):
  File "/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/joblib/parallel.py", line 862, in dispatch_one_batch
    tasks = self._ready_batches.get(block=False)
  File "/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/queue.py", line 168, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tristan/anaconda3/envs/DreamChallenge/lib/python3.10/site-packages/optuna/study/_optimize.p

KeyError: 'handle'